In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

### Question 1

In [2]:
pyspark.__version__

'3.3.2'

In [3]:
spark = SparkSession.builder \
    .master("local[2]") \
    .appName('Spark_Homework') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/28 16:05:41 WARN Utils: Your hostname, DESKTOP-5VNGFU5 resolves to a loopback address: 127.0.1.1; using 172.26.218.188 instead (on interface eth0)
23/02/28 16:05:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 16:05:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Question 2

In [4]:
df = spark.read \
.option('inferSchema', 'true')\
.option('header', 'true')\
.csv('fhvhv_tripdata_2021-06.csv.gz')

In [6]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.DoubleType(), True),
])

In [7]:
df = spark.read \
.schema(schema)\
.option('header', 'true')\
.csv('fhvhv_tripdata_2021-06.csv.gz')

In [8]:
df.repartition(12).write.parquet('data/fhv_tripdata/2021/06/', mode='overwrite')

### Question 3

In [10]:
from pyspark.sql import functions as F
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [18]:
df.createOrReplaceTempView('trips_fhv')

In [25]:
df_result = spark.sql("""
SELECT count(*) from trips_fhv
where date_trunc("Day", pickup_datetime) = '2021-06-15'
""").show()


+--------+
|count(1)|
+--------+
|  452470|
+--------+



### Question $

In [57]:
df_result = spark.sql("""
with trip_time_table as (
    SELECT (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 AS trip_time
    from trips_fhv)

SELECT MAX(trip_time) from trip_time_table

""").show()

+----------------+
|  max(trip_time)|
+----------------+
|66.8788888888889|
+----------------+



In [48]:
from pyspark.sql import types
df \
    .withColumn("DiffInSeconds", F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))\
    .withColumn("DiffInHours",F.col("DiffInSeconds")/3600)\
    .sort(F.col("DiffInHours").desc())\
    .show()   

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|DiffInSeconds|       DiffInHours|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+------------------+
|              B02872|2021-06-25 13:55:41|2021-06-28 08:48:25|          98|         265|      N|                  null|       240764|  66.8788888888889|
|              B02765|2021-06-22 12:09:45|2021-06-23 13:42:44|         188|         198|      N|                  null|        91979|25.549722222222222|
|              B02879|2021-06-27 10:32:29|2021-06-28 06:31:20|          78|         169|      N|                  null|        71931|19.980833333333333|
|              B02800|2021-06-26 22:37:11|2021-06-27 16:49:01|         263|       

### Question 5

### Question 6